# Atmospheric Data Handling Workshop

## 1. Extracting MODIS MCD19A2CMG product data and storing it as dataset.

Clone GitHub Repo

In [ ]:
!git clone https://github.com/msramadan/Atmospheric-Data-Handling-Workshop.git

Install required libraries

In [ ]:
!pip install --upgrade -r https://raw.githubusercontent.com/msramadan/Atmospheric-Data-Handling-Workshop/refs/heads/main/HDF_nc_requirements.txt -q

Import libraries

In [ ]:
import xarray as xr
import rasterio
from pyhdf.SD import SD, SDC
import geopandas as gpd
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import cartopy.crs as ccrs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import os

Here we define a function that extract the study area sub-array from a single HDF

In [ ]:
def hdf_to_2D_array(hdf, lon_bounds, lat_bounds, var_name, cloud_frac_name, cloud_mask):
    """
    hdf: a hdf file read using pyhdf.SD
    lon_bounds: a list containing two values; longitude extracting bounds (small to big).
    lat_bounds: a list containing two values; latitude extracting bounds (big to small).
    var_name: a string represents the name of the variable in the dataset that needs to be extracted.
    cloud_frac_name: a string represents the name of cloud fraction varianble in the dataset.
    cloud_mask: a float of the cloud_mask percentage (eg. 0.01 keeps the values of the pixels with cloud cover percentage lower than 0.01)
    Returns a 2-D array of the extracted data.
    """

    cloud = hdf.select(cloud_frac_name)[:]
    aod = hdf.select(var_name)[:]



    aod_s = aod[int((90-lat_bounds[0])/0.05):int((90-lat_bounds[1])/0.05),int((lon_bounds[0]+180)/.05):int((lon_bounds[1]+180)/.05)]
    aod_s = np.where(aod_s == hdf.select(var_name).attributes()['_FillValue'], np.nan, aod_s)
    aod_s = aod_s * hdf.select(var_name).attributes()['scale_factor']
    cloud_s = cloud[int((90-lat_bounds[0])/0.05):int((90-lat_bounds[1])/0.05),int((lon_bounds[0]+180)/.05):int((lon_bounds[1]+180)/.05)]
    cloud_s = np.where(cloud_s == hdf.select(cloud_frac_name).attributes()['_FillValue'], np.nan, cloud_s)
    cloud_s = cloud_s * hdf.select(cloud_frac_name).attributes()['scale_factor']

    cloud_s[cloud_s > cloud_mask] = np.nan
    cloud_s[cloud_s <= cloud_mask] = 1

    aod_new = np.multiply(aod_s, cloud_s)

    return aod_new

Define the folder directory containing HDF files and files names

In [ ]:
folder = "Atmospheric-Data-Handling-Workshop/Workshop Data/MODIS MCD19A2CMG/"
files = [file for file in os.listdir(folder) if file.endswith('.hdf')]

In [ ]:
files

Now we iterate over all HDF files, extract the a 2-D array of each file, store them in 3-D array, give this array longitude, latitude, and time indices and make it a dataset

In [ ]:
# Define linear spaces for the HDF dimensions
lon = np.arange(-180,180,.05)
lat = np.arange(90,-90,-.05)

# Define lon and lat bounds to be extracted
lons = [34.75, 39.75]
lats = [33.75, 28.75]

# Define Date-Time index that represents the HDF files' Dates
dates = pd.DataFrame(columns=['Name', 'Year', 'Day', 'Date'])
dates['Name'] = files
for i in range(len(dates)):
    dates.loc[i, 'Year'] = dates['Name'][i][12:16]
    dates.loc[i, 'Day'] = dates['Name'][i][16:19]
dts = pd.to_datetime(dates['Year'].astype(str) + dates['Day'].astype(str), format='%Y%j')
dates['Date'] = pd.DatetimeIndex(dts)
dates.sort_values(by='Date', inplace=True)
dates.reset_index(drop=True, inplace=True)

# Define NetCDF indices
nc_dates = dates['Date']
nc_lons = lon[int((lons[0]+180)/.05):int((lons[1]+180)/.05)]
nc_lats = lat[int((90-lats[0])/0.05):int((90-lats[1])/0.05)]

# Define 3-d array of NaN
nc_year = np.zeros((len(nc_dates), len(nc_lats), len(nc_lons))) * np.nan

# Read each file and store its values in the corresponding day in the 3-d array
for i in range(len(dates)):
    ds = SD(folder + dates['Name'][i], SDC.READ)
    nc_year[i] = hdf_to_2D_array(ds, lons, lats, 'AOD_055', 'CloudFraction', 0.01)

# Create NetCDF for the extracted Data
data_set = xr.Dataset(
    {
        'AOD_055': (("time", "latitude", "longitude"), nc_year)
    },
    coords={
        "time": nc_dates,
        "latitude": nc_lats,
        "longitude": nc_lons


    }
    )

In [ ]:
data_set

Import Jordan's borders shapefile and define its projection (WGS84)

In [ ]:
jor = gpd.read_file("Atmospheric-Data-Handling-Workshop/Jordan_Borders_shp/")
jor.set_crs('epsg:4326', inplace=True)

Visualize the dataset

In [ ]:
# Select the 'AOD_055' variable from the dataset
aod_variable = data_set['AOD_055']

# Create a plot for each time step (day)
# Use fig and axes to plot the border on each subplot
g = aod_variable.plot(col='time', col_wrap=5, figsize=(18, 8))

# Add the Jordan border to each subplot
for ax in g.axes.flat:
    jor.plot(ax=ax, facecolor='none', edgecolor='r')

# Display the plot
plt.show()

In [ ]:
!rm -rf /content/Atmospheric-Data-Handling-Workshop

## 2. Spatial resampling and data assimilation techniques for harmonizing multi-source datasets.

Clone GitHub Repo

In [ ]:
!git clone https://github.com/msramadan/Rest-of-Workshop.git

Import required libraries

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
from IPython.display import Image
from shapely.geometry import Polygon
from shapely.geometry import box
import os

import datasets have different spatial and temporal resolutions

In [ ]:
t2m = xr.open_dataset(
    'Rest-of-Workshop/Data/NetCDF4/t2m_workshop.nc'
    ).drop(['number', 'expver']).sel(valid_time=slice(
        '2025-06-22', '2025-06-28'
        ))
data_set = xr.open_dataset(
    'Rest-of-Workshop/Data/Extracted_nc/Extracted_AOD.nc'
    ).sel(time=slice(
        '2025-06-22', '2025-06-28'
        ))

The temperature dataset has hourly data with 0.25 x 0.25 grids

While AOD dataset is daily with 0.05 x 0.05 grids

we need them to have matching resolutions

In [ ]:
t2m

Convert the temperature dataset into daily dataset

In [ ]:
t2m = t2m.coarsen(valid_time=24).mean()
t2m

In [ ]:
t2m['valid_time'] = pd.DatetimeIndex(t2m['valid_time'].dt.date)
t2m

Define the spatial resampling function

In [ ]:
def fine_grid_no_interpolation(cdf, regridding_factor, epsilon = 1e-9):

    """
    cdf: NetCDF4 dataset read by xarray.

    regridding_factor: the factor that we want to resample our dataset by; it has to be a multiplier of the original dimension,
    Eg. using a factor of 5 for 0.25 ERA5 will resample the data into 0.05 degree dataset.

    epsilon: a value to avoid floating point error when defining the new dimensions.

    This function resamples the dataset into finer dimension without interpolation, using Kroniker Product.

    Returns xarray dataset with the new spatial dimensions.
    """

    lons = cdf.variables[[name for name in cdf.coords if 'lon' in name.lower()][0]][:]
    lats = cdf.variables[[name for name in cdf.coords if 'lat' in name.lower()][0]][:]
    var = cdf.variables[list(cdf.data_vars.keys())[0]][:]
    dif_lon = lons[1] - lons[0]
    dif_lat = lats[0] - lats[1]

    lats_fine = np.float64(np.round(np.arange(lats[0] + (dif_lat/2) - (dif_lat/regridding_factor)/2,
                               lats[-1]- (dif_lat/2) + (dif_lat/regridding_factor)/2,
                               -dif_lat/regridding_factor+epsilon),4))

    lons_fine = np.float64(np.round(np.arange(lons[0] - (dif_lon/2) + (dif_lon/regridding_factor)/2,
                               lons[-1]+ (dif_lon/2) - (dif_lon/regridding_factor)/2,
                               dif_lon/regridding_factor-epsilon),4))

    t = cdf[[name for name in cdf.coords if 'time' in name.lower()][0]]
    times = pd.DatetimeIndex(t.to_pandas())

    var_kron = np.kron(var, np.ones((regridding_factor,regridding_factor)))

    var_name = list(cdf.data_vars.keys())[0]

    # Create a DataArray for the variable and then convert to a Dataset
    data_array = xr.DataArray(
        var_kron,
        coords=[times, lats_fine, lons_fine],
        dims=["time", "latitude", "longitude"],
        name=var_name
    )
    data_set = data_array.to_dataset()


    return data_set

In [ ]:
t2m_res = fine_grid_no_interpolation(t2m, 5)

In [ ]:
t2m_res

In [ ]:
Image(url="https://raw.githubusercontent.com/msramadan/Rest-of-Workshop/main/kron1.png", width=800, height=400)

Round the spatial dimension to avoid floating point errors

In [ ]:
def round_coords(ds, decimal=6):
    for coord in ['latitude', 'lat']:
        if coord in ds.coords:
            ds = ds.assign_coords({coord: np.round(ds[coord].values, decimal)})
    for coord in ['longitude', 'lon']:
        if coord in ds.coords:
            ds = ds.assign_coords({coord: np.round(ds[coord].values, decimal)})
    return ds

decimal_places = 6
data_set = round_coords(data_set, decimal_places)
t2m_res = round_coords(t2m_res, decimal_places)

Merged the two datasets

In [ ]:
merged = xr.merge([data_set, t2m_res])
merged

In [ ]:
merged.to_dataframe().dropna(subset='t2m').reset_index().sort_values(by=['latitude'], ascending=False).head(20).round(2)

## 3. Edge-Weighted Averaging

A custom method for estimating time series averages for small polygons situated between grid cells

Define the function

In [ ]:
def extracting_and_averaging_polygon(poly, nc_file, regridding_factor, epsilon = 1e-9):
  poly_4326 = poly.to_crs('epsg:4326')
  x = poly_4326.centroid.x.values[0]
  y = poly_4326.centroid.y.values[0]
  lons = nc_file.variables[[name for name in nc_file.coords if 'lon' in name.lower()][0]][:]
  lats = nc_file.variables[[name for name in nc_file.coords if 'lat' in name.lower()][0]][:]
  dif_lon = lons[1] - lons[0]
  dif_lat = lats[0] - lats[1]

  margin_lon = dif_lon/2 * 1.5
  margin_lat = dif_lat/2 * 1.5

  bounds = poly_4326['geometry'][0].bounds  # Get bounds in EPSG:4326
  maxi = max(max(abs(x - (bounds[0] - margin_lon)), abs(x - (bounds[2] + margin_lon))),
           max(abs(y - (bounds[1] - margin_lat)), abs(y - (bounds[3] + margin_lat))))

  rectangle_polygon_geometry = box(x - maxi - margin_lon,
                                 y - maxi + margin_lat,
                                 x + maxi + margin_lon,
                                 y + maxi - margin_lat)

  # Create the GeoDataFrame
  polygon = gpd.GeoDataFrame(index=[0], crs='epsg:4326', geometry=[rectangle_polygon_geometry])
  nc_file.rio.set_spatial_dims(x_dim="longitude", y_dim="latitude", inplace=True)
  nc_file.rio.write_crs("epsg:4326", inplace=True)
  clipped = nc_file.rio.clip(polygon.geometry,polygon.crs)
  del lons, lats

  # Rename dimensions to match expected names in fine_grid_no_interpolation
  # Assuming the time dimension is named 'valid_time' in the input nc_file (t2m)
  original_time_dim = [name for name in clipped.dims if 'time' in name.lower()][0]
  clipped = clipped.rename({original_time_dim: 'time'})


  data_set = fine_grid_no_interpolation(clipped, regridding_factor, epsilon)
  data_set.rio.set_spatial_dims(x_dim=[name for name in data_set.coords if 'lon' in name.lower()][0],
                                y_dim=[name for name in data_set.coords if 'lat' in name.lower()][0],
                                inplace=True)
  data_set.rio.write_crs("epsg:4326", inplace=True)

  c = data_set.rio.clip(poly.geometry,polygon.crs)

  # Convert to DataFrame and inspect before groupby
  df_before_groupby = c.to_dataframe().dropna().reset_index()


  avg_df = df_before_groupby.groupby('time')[list(c.data_vars.keys())[0]].mean().reset_index()
  return avg_df

Importing a polygon

In [ ]:
gdf = gpd.read_file('Rest-of-Workshop/polygons/Irbid.shp')
gdf.set_crs('epsg:4326', inplace=True) # Set the current CRS

Lets see what the function returns

In [ ]:
extracting_and_averaging_polygon(gdf, t2m, 25)

In [ ]:
raw_ds = t2m
fine_ds = fine_grid_no_interpolation(t2m, 10)

In [ ]:
up = 32.75
down = 32
left = 35.5
right = 36.25
rectangle_polygon_geometry = box(left - .125,
                                 down - .125,
                                 right + .125,
                                 up + .125)

  # Create the GeoDataFrame
polygon = gpd.GeoDataFrame(index=[0], crs='epsg:4326', geometry=[rectangle_polygon_geometry])

raw_ds.rio.set_spatial_dims(x_dim="longitude", y_dim="latitude", inplace=True)
fine_ds.rio.set_spatial_dims(x_dim="longitude", y_dim="latitude", inplace=True)

raw_ds.rio.write_crs("epsg:4326", inplace=True)
fine_ds.rio.write_crs("epsg:4326", inplace=True)

raw_ds1 = raw_ds.rio.clip(polygon.geometry,polygon.crs)
fine_ds1 = fine_ds.rio.clip(gdf.geometry,polygon.crs)



fig, ax = plt.subplots(1, 2, subplot_kw={'projection': ccrs.PlateCarree()}, sharex=True, sharey=True, figsize=(28,10), dpi=100)
timestep = 4
data = raw_ds1["t2m"].isel(valid_time=timestep) - 273.15  # Adjust variable name accordingly
                                                #Here we converted temperature from Kelvin into Celecius
                                                #You can delete the '-273.15' if you are working on different variable

# Extract coordinates
lon = raw_ds1["longitude"]  # Adjust if named differently
lat = raw_ds1["latitude"]

mesh = ax[0].pcolormesh(lon, lat, data, cmap="RdBu_r", shading="auto", edgecolors="white", linewidth=.5)
ax[0].scatter(np.meshgrid(lon, lat)[0], np.meshgrid(lon, lat)[1], color='black', linewidth=0.5)
#cbar = plt.colorbar(mesh, ax=ax[0], orientation="vertical")
#cbar.ax.tick_params(labelsize=17)
ax[0].set_title('Original', size=26)

gridliner = ax[0].gridlines(draw_labels=True, linestyle="None", linewidth=0.0)
gridliner.xformatter = LONGITUDE_FORMATTER
gridliner.yformatter = LATITUDE_FORMATTER


#gridliner.xlocator = mticker.FixedLocator(range(-180, 181, 1))  # Adjust longitude step
#gridliner.ylocator = mticker.FixedLocator(range(-90, 91, 1))    # Adjust latitude step

gridliner.right_labels = False  # Remove right labels
gridliner.top_labels = False    # Remove top labels
gridliner.xlabel_style = {'size': 16, 'rotation':45}
gridliner.ylabel_style = {'size': 16}

data = fine_ds1["t2m"].isel(time=timestep)- 273.15

lon = fine_ds1["longitude"]  # Adjust if named differently
lat = fine_ds1["latitude"]

mesh = ax[1].pcolormesh(lon, lat, data, cmap="RdBu_r", shading="auto", edgecolors="white", linewidth=.5) #try edgecolors="None"
ax[1].set_title('Resampled Clip', size=26)

gridliner = ax[1].gridlines(draw_labels=True, linestyle="None", linewidth=0.0)
gridliner.xformatter = LONGITUDE_FORMATTER
gridliner.yformatter = LATITUDE_FORMATTER
    #gridliner.xlocator = mticker.FixedLocator(range(-180, 181, 10))  # Adjust longitude step
    #gridliner.ylocator = mticker.FixedLocator(range(-90, 91, 10))    # Adjust latitude step
gridliner.right_labels = False  # Remove right labels
gridliner.top_labels = False    # Remove top labels
gridliner.xlabel_style = {'size': 16, 'rotation':45}
gridliner.ylabel_style = {'size': 16}

#cbar = plt.colorbar(mesh, ax=ax[1], orientation="vertical")
#cbar.ax.tick_params(labelsize=17)
gdf.plot(ax=ax[0], edgecolor='k', facecolor='none', linewidth=3)
gdf.plot(ax=ax[1], edgecolor='k', facecolor='none', linewidth=.5)
plt.suptitle(f'Temperature at: {(raw_ds["valid_time"][timestep-1]).dt.date.values}', size=32)
cbar = fig.colorbar(mesh, ax=ax, location='right', shrink=1, aspect=15)
cbar.ax.tick_params(labelsize=15)
cbar.set_label('\n$^o$C', size=22)

## Creating spatially and temporally lagged features for modeling.

In [ ]:
ds = xr.open_dataset('Rest-of-Workshop/Data/NetCDF4/t2m_workshop.nc').drop(['number', 'expver'])

In [ ]:
ds

In [ ]:
x = np.array([[[ 0,  1,  2,  3],
               [ 4,  5,  6,  7],   #Day 1
               [ 8,  9, 10, 11],
               [12, 13, 14, 15]],

              [[16, 17, 18, 19],
               [20, 21, 22, 23],   #Day 2
               [24, 25, 26, 27],
               [28, 29, 30, 31]],

              [[32, 33, 34, 35],
               [36, 37, 38, 39],   #Day 3
               [40, 41, 42, 43],
               [44, 45, 46, 47]]])

In [ ]:
x.flatten()

In [ ]:
s_df = pd.DataFrame(columns=['values'], data=x.flatten())
s_df

Temporal lags

In [ ]:
s_df['values_t-1'] = s_df['values'].shift(len(x[0].flatten()))
s_df['values_t+2'] = s_df['values'].shift(-len(x[0].flatten()))
s_df

In [ ]:
s_df.dropna()

Spatial lag

In [ ]:
col_name = 'values'
len_lons = 4
i = 1
in_df = pd.DataFrame(columns=['values'], data=x.flatten())
in_df[col_name + '_' + f'u{i}'] = in_df[col_name].shift(len_lons * (i))
in_df[col_name + '_' + f'lw{i}'] = in_df[col_name].shift(-len_lons * (i))
in_df[col_name + '_' + f'le{i}'] = in_df[col_name].shift((i))
in_df[col_name + '_' + f'r{i}'] = in_df[col_name].shift(-(i))
in_df[col_name + '_' + f'u-le{i}'] = in_df[col_name].shift(len_lons * (i) + (i))
in_df[col_name + '_' + f'lw-r{i}'] = in_df[col_name].shift(-len_lons * (i) - (i))
in_df[col_name + '_' + f'u-r{i}'] = in_df[col_name].shift(len_lons * (i) - (i))
in_df[col_name + '_' + f'lw-le{i}'] = in_df[col_name].shift(-len_lons * (i) + (i))

In [ ]:
in_df

In [ ]:
print(x)
in_df.dropna()

Define time lag generating function

In [ ]:
def time_lag_features(in_df, col_name, lag_steps):
  """
    in_df: input df that has the feature that we want to create temporally lagged feature for
    col_name: a string representing the column name of the feature
    lag_steps: how many lagged steps you want to generate (in both directions)
  """
  lag = len(in_df[in_df['valid_time'] == in_df['valid_time'].unique()[0]])
  for i in range(lag_steps):
    in_df[col_name + '_' + f'lag{i+1}'] = in_df[col_name].shift(lag * (i+1))
    in_df[col_name + '_' + f'lag-{i+1}'] = in_df[col_name].shift(lag * -(i+1)) # hash this line if you dont want the procceeding time teps
  return in_df

Define spatial lag features generating function

In [ ]:
def spatial_lag_features(in_df, col_name, shift_list):
    """
    in_df: dataframe that contains the target column.
    col_name: name of the target column.
    shift_list: list of shift values to generate.
    """

    len_lons = len(in_df['longitude'].unique())
    for i in shift_list:
        in_df[col_name + '_' + f'u{i}'] = in_df[col_name].shift(len_lons * (i))
        in_df[col_name + '_' + f'lw{i}'] = in_df[col_name].shift(-len_lons * (i))
        in_df[col_name + '_' + f'le{i}'] = in_df[col_name].shift((i))
        in_df[col_name + '_' + f'r{i}'] = in_df[col_name].shift(-(i))
        in_df[col_name + '_' + f'u-le{i}'] = in_df[col_name].shift(len_lons * (i) + (i))
        in_df[col_name + '_' + f'lw-r{i}'] = in_df[col_name].shift(-len_lons * (i) - (i))
        in_df[col_name + '_' + f'u-r{i}'] = in_df[col_name].shift(len_lons * (i) - (i))
        in_df[col_name + '_' + f'lw-le{i}'] = in_df[col_name].shift(-len_lons * (i) + (i))
    return in_df

In [ ]:
df = ds.to_dataframe().reset_index()
df

In [ ]:
df.sort_values(['valid_time', 'latitude', 'longitude'], ascending=[True, False, True], inplace=True)

In [ ]:
np.unique(ds['t2m'].values.flatten() == df['t2m'].values)

In [ ]:
prac = ds.to_dataframe().reset_index()

In [ ]:
prac = time_lag_features(prac, 't2m', 2)
prac = spatial_lag_features(prac, prac.columns[3:], [1, 2, 3])

In [ ]:
prac